In [3]:
# prompt: drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install -q --upgrade transformers datasets accelerate peft bitsandbytes trl


In [ ]:
import pandas as pd
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from datasets import load_dataset, Dataset
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    IntervalStrategy,
    BitsAndBytesConfig
)
from huggingface_hub import notebook_login

# Ensure PEFT imports
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
# Authenticate to Hugging Face interactively
notebook_login()


In [ ]:
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from sklearn.model_selection import train_test_split
SEED = 42

balanced_wine_reviews = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/wine_recommendation/balanced_wine_reviews.csv')
encoder = LabelEncoder()
balanced_wine_reviews['variety'] = encoder.fit_transform(balanced_wine_reviews['variety'])

train_texts, test_texts, train_labels, test_labels = train_test_split(
    balanced_wine_reviews['description'], balanced_wine_reviews['variety'], test_size=0.8, random_state=SEED, stratify=balanced_wine_reviews['variety']
)

train_dataset = Dataset.from_dict({'description': train_texts, 'variety': train_labels})
test_dataset = Dataset.from_dict({'description': test_texts, 'variety': test_labels})

In [ ]:
# 5. Load tokenizer
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Ensure padding token is defined
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    num_labels=3
)
# Reduce memory by checkpointing
model.gradient_checkpointing_enable()
# Ensure pad token ID
model.config.pad_token_id = tokenizer.eos_token_id

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
# 6. Prepare model for k-bit training and apply LoRA
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    task_type="SEQ_CLS"
)
# now apply PEFT
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [ ]:
def tokenize_function(examples):
    """
    Μετατρέπει τα κείμενα σε tokens χρησιμοποιώντας τον tokenizer του προεκπαιδευμένου μοντέλου.

    Κάθε κείμενο μετατρέπεται σε ακολουθία tokens με padding μέχρι το μέγιστο μήκος (max_length)
    και με περικοπή (truncation) ώστε να μην ξεπερνά το όριο των 512 tokens.

    :param examples: Λεξικό που περιέχει το κείμενο υπό το κλειδί 'text'.
    :return: Λεξικό με τα tokenized αποτελέσματα.
    """
    return tokenizer(
        examples['description'],
        padding="max_length",  # Προσθήκη padding ώστε όλα τα sequences να έχουν το ίδιο μήκος.
        truncation=True,       # Ενεργοποίηση του truncation για να περιοριστούν τα sequences στο max_length.
        max_length=512,        # Ορισμός μέγιστου μήκους ακολουθίας.
    )

# Εφαρμογή της συνάρτησης tokenization στο σύνολο εκπαίδευσης και δοκιμής σε παρτίδες.
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Μετονομασία της στήλης "variety" σε "labels", όπως απαιτείται από το μοντέλο της Hugging Face.
train_dataset = train_dataset.rename_column("variety", "labels")
test_dataset = test_dataset.rename_column("variety", "labels")

# Ορισμός του format των δεδομένων σε PyTorch tensors για να μπορούν να χρησιμοποιηθούν από το μοντέλο.
train_dataset.set_format("torch")
test_dataset.set_format("torch")

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
# Define compute_metrics function

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="macro")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# Ορισμός του callback για early stopping, ώστε να σταματήσει η εκπαίδευση εάν δεν υπάρχει βελτίωση.
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,             # Αριθμός εποχών χωρίς βελτίωση πριν τη διακοπή της εκπαίδευσης.
    early_stopping_threshold=0.001         # Κατώφλι βελτίωσης που πρέπει να επιτευχθεί για να θεωρηθεί ότι υπάρχει πρόοδος.
)

In [ ]:

# 10. Training arguments
training_args = TrainingArguments(
    output_dir="./lora_results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=5,
    learning_rate=1e-4,
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

In [ ]:
# 11. Trainer setup and training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
# 9. Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: savsko08 (savsko08-university-of-patras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
results = trainer.evaluate()
print(results)

In [ ]:

# After training:
model.save_pretrained('/content/drive/MyDrive/llama2-wines-finetuned')
tokenizer.save_pretrained('/content/drive/MyDrive/llama2-wines-finetuned')

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Πρόβλεψη στο test set χρησιμοποιώντας τον εκπαιδευμένο trainer.
predictions = trainer.predict(test_dataset)

# Εξαγωγή των αληθινών ετικετών (labels) από το test_dataset.
y_true = np.array([example['labels'] for example in test_dataset])

# Εξαγωγή των προβλεπόμενων ετικετών: επιλέγουμε την κλάση με τη μεγαλύτερη τιμή πιθανοτήτων για κάθε δείγμα.
y_pred = np.argmax(predictions.predictions, axis=1)


# Εκτύπωση του Classification Report για την αξιολόγηση της απόδοσης του μοντέλου.
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=encoder.classes_))